# Introduction to _pyspark_

[Spark][spark], like Hadoop itself, is a framework for programming with an abstraction of the map-reduce paradigm. Its main data structure (RDD) allows better utilization of the memory of the nodes, and this made it very popular in recent years. Spark was originally part of the Hadoop ecosystem, however it was so useful, that eventually it was decided to make it available as a stand-alone framework. Spark is written in [Scala][scala], but it suports APIs for Java, R and of course Python.

Spark is made of 5 building blocks:

* Spark core - the fundamentals components of the language. It provides distributed task dispatching, scheduling, and basic I/O functionalities, exposed through an API centered on the RDD abstraction.
* Spark SQL - tools for working with DataFrames. It provides an API for embedding SQL scripts, as well as connections with an ODBC/JDBC server.
* Spark streaming - facilitates tasks witha a data stream. It ingests data in mini-batches and performs RDD transformations on those mini-batches of data.
* Spark MLlib - distributed versions of various machine learning (ML) algorithms.
* Spark GraphX - graph processing framework.

In our course, we will explore 3 of the 5 - Spark core, Spark SQL and Spark MLlib, and we will do it using the Python API - **pyspark**.

[spark]: https://en.wikipedia.org/wiki/Apache_Spark "Apache Spark - Wikipedia"
[scala]: https://en.wikipedia.org/wiki/Scala_(programming_language) "Scala - Wikipedia"

In [1]:
import os, sys, json, collections, itertools
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

local_dir = "file:///{d}/".format(d=os.getcwd())

if 'sc' not in globals():
    conf = SparkConf().setAppName('appName').setMaster('local')
    sc = SparkContext(conf=conf)
    spark = SparkSession(sc)

# Setting spark on your machine

1. Install Java8: `brew cask install adoptopenjdk/openjdk/adoptopenjdk8`
1. Install scala: `brew install scala`
1. Install apache-spark: `brew install apache-spark`
1. Set these enviroment variables to your python interpreter path: `PYSPARK_PYTHON` and `PYSPARK_DRIVER_PYTHON`
1. Set `JAVA_HOME` to the path of your java installation `export JAVA_HOME=$(/usr/libexec/java_home -v 1.8)`
1. Install pyspark: `python3 -m install pyspark`


## RDD
**Resilient Distributed Dataset (RDD)** is the main data object in Spark and it is an abstraction of the data parallelization. This means that we can work with a single RDD, where in fact its data, as well as its processing, may be distributed in the cluster.

Data sharing is slow in MapReduce due to replication, serialization, and disk IO (Actually, most Hadoop applications spend more than 90% of the time doing HDFS read-write operations.). Recognizing this problem, RDDs support **in-memory** processing computation. This means, it stores the state of memory as an object across the jobs and the object is sharable between those jobs.

Two technical comments:

* RDDs are immutable, which has a great influence on the appearence of Spark code.
* If the elements of an RDD are tuples (which is a Spark data type, equivalent to Python tuples of length 2), then each tuple is automatically recognized as a pair of a **key** and a **value**.

### Transformations vs Actions
RDD **transformations** are operations applied on RDDs to yield a new RDD. On the other hand, **actions** are operations applied on RDDs to yield a non-RDD result (number, string, list, etc.). 

Here are some examples:

* Transformations:
    * _map(func)_ - Returns a new distributed dataset, formed by passing each element of the source through a function func.
    * _flatMap(func)_ - Similar to map, but each input item can be mapped to 0 or more output items (so func should return a Seq rather than a single item).
    * _filter(func)_ - Returns a new dataset formed by selecting those elements of the source on which func returns true.
    * _union(otherDataset)_ - Returns a new dataset that contains the union of the elements in the source dataset and the argument.
    * _groupByKey()_ - When called on a dataset of (K, V) pairs, returns a dataset of (K, Iterable(V)) pairs.
    * _reduceByKey(func)_ - When called on a dataset of (K, V) pairs, returns a dataset of (K, V) pairs where the values for each key are aggregated using the given reduce function func, which must be of type (V, V) ⇒ V.
    * _sortByKey([ascending])_ - When called on a dataset of (K, V) pairs where K implements Ordered, returns a dataset of (K, V) pairs sorted by keys in ascending or descending order, as specified in the Boolean ascending argument.
* Actions:
    * _reduce(func)_ - Aggregate the elements of the dataset using a function func (which takes two arguments and returns one). The function should be commutative and associative so that it can be computed correctly in parallel.
    * _count()_ - Returns the number of elements in the dataset.
    * _take(n)_ - Returns an array with the first n elements of the dataset. 
    * _saveAsTextFile(path)_ - Writes the elements of the dataset as a text file (or set of text files) in a given directory in the local filesystem, HDFS or any other Hadoop-supported file system. Spark calls _toString()_ on each element to convert it to a line of text in the file.

Two technical comments:

* In most cases one applies a chain of transformations which ends with an action. Each RDD in such dependency chain has a pointer (dependency) to its parent RDD. Spark is **lazy**, so nothing will be executed until an action will trigger the chain. Therefore, RDD transformation is not a set of data but is a step in a program (might be the only step) telling Spark how to get data and what to do with it.
* Spark is written in Scala, which does not support some of the functionalities of Python. This is why the Python API offers some additional transformations which are not part of the core functionalities, but a wrapper of them. 

## Example 1 - RDD fundamentals

In [2]:
words = sc\
    .textFile("melville-moby_dick.txt")\
    .flatMap(lambda line: line.split())\
    .filter(lambda word: word.isalpha())\
    .map(lambda word: word.lower())
words.take(5)

['dick', 'by', 'herman', 'melville', 'by']

### Question 1

How may words are in the book? (words contain only letters)

In [3]:
words.count()

171870

### Question 2

How many _unique_ words are in the book?

In [4]:
unique_words = words.groupBy(lambda word: word)
unique_words.take(5)

[('dick', <pyspark.resultiterable.ResultIterable at 0x10d54bf90>),
 ('by', <pyspark.resultiterable.ResultIterable at 0x10d54be90>),
 ('herman', <pyspark.resultiterable.ResultIterable at 0x10d568e50>),
 ('melville', <pyspark.resultiterable.ResultIterable at 0x10d54d810>),
 ('a', <pyspark.resultiterable.ResultIterable at 0x10d54bfd0>)]

In [5]:
unique_words.count()

13739

### Question 3

What is the most common word in the book?

In [8]:
word_count = unique_words\
    .mapValues(lambda group: len(group))\
    .sortBy(lambda word_count: word_count[1], ascending=False)
print (word_count.take(10))

[('the', 14226), ('of', 6545), ('and', 6238), ('a', 4597), ('to', 4518), ('in', 4058), ('that', 2744), ('his', 2485), ('it', 1765), ('i', 1724)]


### Question 4

What is the most common word in the book which is not a [stop-word][1]? (a file with the English stop-words is available in the folder)

[1]: https://en.wikipedia.org/wiki/Stop_words "Stop words - Wikipedia"

In [7]:
stop_words = sc\
    .textFile("english words.txt")\
    .map(lambda word: (word, 1))
print (stop_words.take(10))

[('A', 1), ('a', 1), ('aa', 1), ('aal', 1), ('aalii', 1), ('aam', 1), ('Aani', 1), ('aardvark', 1), ('aardwolf', 1), ('Aaron', 1)]


In [19]:
word_count_2 = word_count\
    .subtractByKey(stop_words)\
    .sortBy(lambda word_count: word_count[1], ascending=False)
print (word_count_2.take(5))

## Exercise 1:
Read the file "english words" into an RDD and answer the following questions:
1. How many words are listed in the file?
1. What is the most common first letter?
1. What is the longest word in the file?
1. How many words include all 5 vowels?

Spark and its related packages are constantly changing, and even the most basic scripts may become unusable from version to version. Threfore it is a good idea to be familiar with the documentation part, which is (trying to be) updated and helpful. Here are some relevant documentation links:

* [Spark 2.0.2][spark]
    * General concepts
        * [Programing guide][pg]
        * [Data structures][ds] - this includes explanations about DataFrames, DataSets and SQL
    * Python API
        * [pyspark package][pyspark] - this includes the [SparkConf][conf], [SparkContext][sc] and [RDD][rdd] classes
        * [pyspark.sql module][sql] - this includes the [SparkSession][ss], [DataFrame][df], [Row][row] and [Column][col] classes

[spark]: https://spark.apache.org/docs/2.0.2/index.html "Spark 2.0.2"
[pg]: https://spark.apache.org/docs/2.0.2/programming-guide.html "Spark programming guide"
[ds]: https://spark.apache.org/docs/2.0.2/sql-programming-guide.html "Data structures programming guide"
[pyspark]: https://spark.apache.org/docs/2.0.2/api/python/index.html "pyspark"
[conf]: https://spark.apache.org/docs/2.0.2/api/python/pyspark.html#pyspark.SparkConf "SparkConf"
[sc]: https://spark.apache.org/docs/2.0.2/api/python/pyspark.html#pyspark.SparkContext "SparkContext"
[rdd]: https://spark.apache.org/docs/2.0.2/api/python/pyspark.html#pyspark.RDD "RDD"
[sql]: https://spark.apache.org/docs/2.0.2/api/python/pyspark.sql.html "pyspark.sql module"
[ss]: https://spark.apache.org/docs/2.0.2/api/python/pyspark.sql.html#pyspark.sql.SparkSession "SparkSession"
[df]: https://spark.apache.org/docs/2.0.2/api/python/pyspark.sql.html#pyspark.sql.DataFrame "DataFrame"
[row]: https://spark.apache.org/docs/2.0.2/api/python/pyspark.sql.html#pyspark.sql.Row "Row"
[col]: https://spark.apache.org/docs/2.0.2/api/python/pyspark.sql.html#pyspark.sql.Column "Column"